In [2]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
from utils import *

In [3]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '5625f73035eb4558b9e6bcddea45edda',
}

########### get reporting_economy #############


In [3]:
params = urllib.parse.urlencode({
    # Request parameters
    'ig': 'individual',
    'reg': 'all',
    'gp': 'all',
    'lang': '1',
})

try:
    conn = http.client.HTTPSConnection('api.wto.org')
    conn.request("GET", "/timeseries/v1/reporters?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data[0:100])
    json_data=data.decode('utf8')
    data=json.loads(json_data)
    
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



b'[{"code":"004","iso3A":"AFG","name":"Afghanistan","displayOrder":10},{"code":"008","iso3A":"ALB","na'


In [9]:
type(data[0]['code'])

str

In [3]:
countries_code=getDF(data,dtype=str)
countries_code.to_csv('./data/countries_code.csv',index=False)

NameError: name 'data' is not defined

### 获取 双边贸易数据###

In [4]:
countries_code=pd.read_csv('./data/countries_code.csv',dtype=str)
countries_code.head()

,code,iso3A,name,displayOrder
0,004,AFG,Afghanistan,10
1,008,ALB,Albania,40
2,012,DZA,Algeria,50
3,016,ASM,American Samoa,70
4,020,AND,Andorra,90


In [5]:
reporting_economies=countries_code['code'].tolist()

reporting_economies_str=""
for i in reporting_economies:
    reporting_economies_str+=i+","
reporting_economies_str=reporting_economies_str[0:-1]


In [6]:
partner_economies=countries_code['code'].tolist()
partner_economies_str=""
for i in partner_economies:
    if i in ['532','666']:
        continue
    partner_economies_str+=i+","
partner_economies_str=partner_economies_str[0:-1]


In [ ]:
timePeriods=['2015']
for timePeriod in timePeriods:
    print(f'getting {timePeriod} data')
    data=get_data(timePeriod,headers,reporting_economies_str,partner_economies_str,step=10000)
    df=getDF(data,dtype=str)
    df.to_csv(f'./data/data_{timePeriod}.csv',index=False)

In [9]:
def reconnect(timePeriods,offset,step,end):
    try:
        for timePeriod in timePeriods:
            print(f'getting {timePeriod} data')
            data=get_data(timePeriod,headers,reporting_economies_str,partner_economies_str,offset=offset,step=step,end=end)
            df=getDF(data,dtype=str)
            df.to_csv(f'./data/data_json/data_{timePeriod}_{offset}_{end}.csv',index=False)
    except Exception as e:
        reconnect(timePeriods,offset,step,end)

In [11]:
##网络不好时则分段传输
timePeriods=['2015']
offset=0
step=5000
total_end=145000
while offset<=total_end:
    end=offset+step
    reconnect(timePeriods,offset,step,end)
    offset+=step

getting 2015 data
getting 2015 data14...     	 
getting 2015 data42614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
[Errno 104] Connection reset by peer
getting 2015 data
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data142614...     	 
getting 2015 data/142614...     	 
getting 2015 data/142614...     	 
getting 2015 data/142614...     	 
getting 2015 data/142614...     	 
getting 2015 data/142614...     	 
getting 2015 data/142614...     	 
getting 2015 data/142614...     	 
getting

In [27]:
timePeriods=['2017']
for i in range(8,14):
    offset=i*10000
    step=5000
    end=(i+1)*10000
    reconnect(timePeriods,offset,step,end)

getting 2017 data
[Errno 104] Connection reset by peer
getting 2017 data
getting 2017 data125204...     	 
getting 2017 data125204...     	 
getting 2017 data/125204...     	 
getting 2017 data/125204...     	 
getting 2017 data/125204...     	 


## 获取products/sectors info

In [7]:
params = urllib.parse.urlencode({
    # Request parameters
    'pc': 'MT2',
    'lang': '1',
})

try:
    conn = http.client.HTTPSConnection('api.wto.org')
    conn.request("GET", "/timeseries/v1/products?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    json_data=data.decode('utf8')
    data=json.loads(json_data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

df=getDF(data,dtype=str)
df.to_csv('./data/ProductOrSectorCode_detail.csv',index=False)

In [8]:
df.head()

,code,name,note,productClassification,codeUnique,displayOrder,hierarchy
0,01,Animal products,None,MT2,MT2 - 01 -,None,01
1,02,Dairy products,None,MT2,MT2 - 02 -,None,02
2,03,"Fruits, vegetables, plants",None,MT2,MT2 - 03 -,None,03
3,04,"Coffee, tea",None,MT2,MT2 - 04 -,None,04
4,05,Cereals and preparations,None,MT2,MT2 - 05 -,None,05


###获取国家坐标

In [16]:
iso3A_to2A=pd.read_csv('./lib/wikipedia-iso-country-codes.csv')
iso3A_to2A=iso3A_to2A.loc[:,['Alpha-2 code','Alpha-3 code']]
iso3A_to2A.set_index('Alpha-2 code',inplace=True)
iso3A_to2A=iso3A_to2A.to_dict()['Alpha-3 code']

iso2A_to_location=pd.read_csv('./lib/countries.csv')
iso2A_to_location['iso3A']=0

d={}
for i in range(0,len(iso2A_to_location)):
    iso2A=iso2A_to_location['country'][i]
    if iso3A_to2A.get(iso2A):
        iso_3A=iso3A_to2A[iso2A]
        latitude=iso2A_to_location['latitude'][i]
        longitude=iso2A_to_location['longitude'][i]
        d[iso_3A]={'latitude':latitude,'longitude':longitude}


In [17]:
d['AND']

{'latitude': 42.546245, 'longitude': 1.601554}

In [18]:
country_codes=pd.read_csv('./data/countries_code.csv',dtype=str)
country_codes.drop(['displayOrder'],inplace=True,axis=1)
country_codes['latitude']=np.nan
country_codes['longitude']=np.nan
for i in range(0,len(country_codes)):
    iso3A=country_codes['iso3A'][i]
    if d.get(iso3A):
        country_codes.loc[i,'latitude']=d[iso3A]['latitude']
        country_codes.loc[i,'longitude']=d[iso3A]['longitude']

In [19]:
country_codes.head()

,code,iso3A,name,latitude,longitude
0,004,AFG,Afghanistan,33.939110,67.709953
1,008,ALB,Albania,41.153332,20.168331
2,012,DZA,Algeria,28.033886,1.659626
3,016,ASM,American Samoa,-14.270972,-170.132217
4,020,AND,Andorra,42.546245,1.601554


In [20]:
empty=country_codes[country_codes.isnull().T.any()]
empty

,code,iso3A,name,latitude,longitude
20,058,BLX,Belgium-Luxembourg,NaN,NaN
50,531,CUW,Curaçao,NaN,NaN
66,918,EEC,European Union,NaN,NaN
142,532,NaN,Netherlands Antilles (incl. Aruba),NaN,NaN
164,642,ROM,Romania,NaN,NaN
169,534,SXM,Saint Martin,NaN,NaN
185,728,SSD,South Sudan,NaN,NaN
193,158,CHT,Chinese Taipei,NaN,NaN
212,888,NaN,UNMIK/Kosovo,NaN,NaN


In [21]:
e_index=empty.index.to_list()
e_index

[20, 50, 66, 142, 164, 169, 185, 193, 212]

In [22]:
##手动查询
location=[
    [50.0547,5.4677],
    [12.1696,68.99],
    [53,9],
    [12.5211,69.9683],
    [45.9432,24.9668],#Romania
    [18.0718,63.0501],#Saint Martin	
    [6.8770,31.3070],#South Sudan
    [23.6978,120.9605],#Chinese Taipei
    [42.6026,20.9030]

]

In [23]:
country_codes.loc[e_index,['latitude','longitude']]=location
country_codes.iloc[e_index,:]

,code,iso3A,name,latitude,longitude
20,058,BLX,Belgium-Luxembourg,50.0547,5.4677
50,531,CUW,Curaçao,12.1696,68.9900
66,918,EEC,European Union,53.0000,9.0000
142,532,NaN,Netherlands Antilles (incl. Aruba),12.5211,69.9683
164,642,ROM,Romania,45.9432,24.9668
169,534,SXM,Saint Martin,18.0718,63.0501
185,728,SSD,South Sudan,6.8770,31.3070
193,158,CHT,Chinese Taipei,23.6978,120.9605
212,888,NaN,UNMIK/Kosovo,42.6026,20.9030


In [24]:
country_codes.to_csv('./data/countries_info.csv',index=False)